# Подготовка данных
Необходимо создать excel или использовать готовую таблицу с некоторыми тестовыми данными

In [1]:
import pandas as pd
import math
import time

df = pd.read_excel("tests6.xlsx")
df.head()

,Id,Direction,Section,TestCaseName,Automated,Preconditions,Steps,Postconditions,ExpectedResult,TestData,Comments,Iterations,Priority,State,CreatedDate,CreatedById,Tags
0,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Highest,Ready,05/23/2024 14:48:49,Служебный пользователь для Автотестов,смоук хф
1,NaN,NaN,NaN,NaN,NaN,"Роль:\n\n\nОперационист ТП, Контролер",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,Выполнен вход в ЕФР под ролью Контролер,NaN,NaN,Открывается страница поиска клиента,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,Предварительно инициировать/найти операцию дос...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,"Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,На странице задач отображается список задач(по...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Начало обработки

Удаляем ненужные столбцы, заполняем нужные ячейки дублирующими значениями

In [2]:
start_timer = time.time()

def clean_dataset(dataset):
    output_dataset = dataset[["Id", "Direction", "Section", "TestCaseName", "Preconditions", "Steps", "Postconditions", "ExpectedResult"]].copy()
    output_dataset[["Id", "Direction", "Section", "TestCaseName"]] = output_dataset[["Id", "Direction", "Section", "TestCaseName"]].ffill()
    return output_dataset

output = clean_dataset(df)
end_timer = time.time()

print(end_timer - start_timer)
output.to_excel("output.xlsx", index = False)
output.head()

0.005433559417724609


,Id,Direction,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult
0,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,NaN,NaN,NaN,NaN
1,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,"Роль:\n\n\nОперационист ТП, Контролер",NaN,NaN,NaN
2,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,Выполнен вход в ЕФР под ролью Контролер,NaN,NaN,Открывается страница поиска клиента
3,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,Предварительно инициировать/найти операцию дос...,NaN,NaN,NaN
4,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,На странице задач отображается список задач(по...


# Разбиваем тест-кейсы на отдельные DataFrame

На выходе получим массив датафреймов(тест-кейсов)

In [3]:
start_timer = time.time()

def parse_tests_by_id(dataset):
    test_cases = []
    for group_name, frame in dataset.groupby('Id'):
        test_cases.append(frame)

    return test_cases

output_array = parse_tests_by_id(output)
end_timer = time.time()
print(end_timer - start_timer)
output_array[0]

0.003037691116333008


,Id,Direction,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult
0,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,NaN,NaN,NaN,NaN
1,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,"Роль:\n\n\nОперационист ТП, Контролер",NaN,NaN,NaN
2,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,Выполнен вход в ЕФР под ролью Контролер,NaN,NaN,Открывается страница поиска клиента
3,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,Предварительно инициировать/найти операцию дос...,NaN,NaN,NaN
4,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,На странице задач отображается список задач(по...
5,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Нажать ""Взять"" для задачи из пула текущего пол...",NaN,Задаче назначается исполнитель – текущий польз...
6,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Кликнуть ""Мои задачи""",NaN,"Открывается пул задач, доступных на исполнение"
7,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Нажать ""Исполнить"" для задачи из пула текущего...",NaN,Открывается форма задачи в текущей вкладке


In [4]:
output_array[1]

,Id,Direction,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult
8,544032.0,AI_Отладка,Пул задач пользователя,Контролер. Взять и исполнить задачу,NaN,NaN,NaN,NaN
9,544032.0,AI_Отладка,Пул задач пользователя,Контролер. Взять и исполнить задачу,Роль:\n\n\nОперационист ТП,NaN,NaN,NaN
10,544032.0,AI_Отладка,Пул задач пользователя,Контролер. Взять и исполнить задачу,Выполнен вход в ЕФР,NaN,NaN,Открывается страница поиска клиента
11,544032.0,AI_Отладка,Пул задач пользователя,Контролер. Взять и исполнить задачу,NaN,"Кликнуть ""Задачи"" в верхней части экрана",NaN,На форме отображается список задач доступных н...
12,544032.0,AI_Отладка,Пул задач пользователя,Контролер. Взять и исполнить задачу,NaN,"Нажать ""Взять и исполнить"" для задачи из пула ...",NaN,открывается форма задачи


In [5]:
#output_array[2]

# Удаление пустых ячеек

In [6]:
start_timer = time.time()

def remove_empty_cells(array):
    # обрабатываем "Preconditions", "Steps", "Postconditions", "ExpectedResult" (удаляем первую ячейку)
    def up_cells(tst_case, column):
        for i in range(len(tst_case[column])):
            if i == len(tst_case.index) - 1:
                tst_case.at[tst_case.index[i], column] = math.nan
                break
            else:
                tst_case.at[tst_case.index[i], column] = tst_case.at[tst_case.index[i + 1], column]
                
        return tst_case
        
    for dframe in array:
        for col in ["Preconditions", "Steps", "Postconditions", "ExpectedResult"]:
            dframe = up_cells(dframe, col)
              
        dframe["Steps"] = dframe["Steps"].fillna(dframe["Preconditions"])
        dframe["Steps"] = dframe["Steps"].fillna(dframe["Postconditions"])
        dframe.drop(["Preconditions", "Postconditions"], inplace=True, axis=1)     
            
    return array

edited_output_array = remove_empty_cells(output_array)
end_timer = time.time()
print(end_timer - start_timer)
edited_output_array[0]

0.027859926223754883


,Id,Direction,Section,TestCaseName,Steps,ExpectedResult
0,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,"Роль:\n\n\nОперационист ТП, Контролер",NaN
1,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,Выполнен вход в ЕФР под ролью Контролер,Открывается страница поиска клиента
2,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,Предварительно инициировать/найти операцию дос...,NaN
3,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,"Кликнуть на раздел ""Задачи"" в верхней части эк...",На странице задач отображается список задач(по...
4,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,"Нажать ""Взять"" для задачи из пула текущего пол...",Задаче назначается исполнитель – текущий польз...
5,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,"Кликнуть ""Мои задачи""","Открывается пул задач, доступных на исполнение"
6,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,"Нажать ""Исполнить"" для задачи из пула текущего...",Открывается форма задачи в текущей вкладке
7,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,NaN,NaN


In [7]:
pd.DataFrame(pd.concat(edited_output_array)).to_excel("edited_output.xlsx", index=False)

In [8]:
edited_output_array[1]

,Id,Direction,Section,TestCaseName,Steps,ExpectedResult
8,544032.0,AI_Отладка,Пул задач пользователя,Контролер. Взять и исполнить задачу,Роль:\n\n\nОперационист ТП,NaN
9,544032.0,AI_Отладка,Пул задач пользователя,Контролер. Взять и исполнить задачу,Выполнен вход в ЕФР,Открывается страница поиска клиента
10,544032.0,AI_Отладка,Пул задач пользователя,Контролер. Взять и исполнить задачу,"Кликнуть ""Задачи"" в верхней части экрана",На форме отображается список задач доступных н...
11,544032.0,AI_Отладка,Пул задач пользователя,Контролер. Взять и исполнить задачу,"Нажать ""Взять и исполнить"" для задачи из пула ...",открывается форма задачи
12,544032.0,AI_Отладка,Пул задач пользователя,Контролер. Взять и исполнить задачу,NaN,NaN


# (Опционально) Замена тест-кейсов в массиве на наборы текстовых предложений

In [9]:
# Предложения для теста

# Во время обучения модели machine learning, важно обеспечить достаточное количество данных для тренировки
# Использование API позволяет интегрировать различные сервисы в ваше приложение, упрощая процесс разработки
# Для анализа больших объемов данных рекомендуется использовать фреймворк Hadoop, который обеспечивает эффективную обработку данных
# В процессе дебаггинга программы была обнаружена ошибка, связанная с неправильным использованием переменных
# При разработке user interface необходимо учитывать принципы user experience, чтобы сделать приложение максимально удобным для пользователя

def switch_to_lang(array):
    array.clear()
    array.append("Во время обучения модели machine learning, важно обеспечить достаточное количество данных для тренировки") 
    array.append("Во время обучения модели машинного обучения, важно обеспечить достаточное количество данных для тренировки") 
    array.append("Использование API позволяет интегрировать различные сервисы в ваше приложение, упрощая процесс разработки") 
    array.append("The use of an API allows you to integrate various services into your application, simplifying the development process")
    array.append("Для анализа больших объемов данных рекомендуется использовать фреймворк Hadoop, который обеспечивает эффективную обработку данных") 
    array.append("В процессе дебаггинга программы была обнаружена ошибка, связанная с неправильным использованием переменных") 
    array.append("При разработке user interface необходимо учитывать принципы user experience, чтобы сделать приложение максимально удобным для пользователя") 
    array.append("При разработке пользовательского интерфейса необходимо учитывать принципы пользовательского опыта, чтобы сделать приложение максимально удобным для пользователя") 
    return array

edited_output_array = switch_to_lang(edited_output_array)

len(edited_output_array)

8

# spaCy

```conda install spacy```

```python -m spacy download ru_core_news_lg```


In [15]:
import spacy
import torch

import torch.nn.functional as F

start_timer = time.time()

spacy_nlp = spacy.load("ru_core_news_lg")

def vectorize_text(text: str, spacy_model: spacy.Language) -> torch.Tensor:
    doc = spacy_model(text)
    return torch.from_numpy(doc.tensor)

spacy_vectorized_output_array = []

for test_case in edited_output_array:
    spacy_vectorized_output_array.append(F.normalize(vectorize_text(str(test_case), spacy_nlp), p=1, dim=1))

def output_spacy_to_csv():
    for i in range(len(spacy_vectorized_output_array)):
        pd.DataFrame(spacy_vectorized_output_array[i]).to_csv("spacy_output/spacy_vectorized_output_" + str(i) + ".csv", index = False)    

output_spacy_to_csv()
    
end_timer = time.time()
print(end_timer - start_timer)
spacy_vectorized_output_array[0]

2.550997495651245


tensor([[ 2.1458e-02,  1.2491e-03, -1.1220e-03,  ..., -9.1945e-03,
         -3.3335e-05,  2.8552e-02],
        [-4.8291e-03, -2.2108e-03,  1.0211e-02,  ...,  5.2530e-03,
         -5.3169e-03, -3.9651e-03],
        [-8.5779e-03, -4.8351e-03, -4.0841e-03,  ...,  1.9009e-04,
          4.8387e-03,  1.0281e-02],
        ...,
        [-3.0558e-03,  2.1174e-02,  1.0701e-02,  ..., -9.2181e-03,
          1.0107e-02,  2.9649e-03],
        [-1.7001e-02, -2.0753e-02,  5.6611e-04,  ..., -5.6224e-03,
          4.5322e-03, -2.5375e-02],
        [-1.2661e-02, -7.4628e-03, -1.7357e-02,  ..., -8.1719e-03,
         -9.7612e-03, -1.1799e-02]])

In [ ]:
spacy_vectorized_output_array[1]

In [ ]:
spacy_vectorized_output_array[2]

# sentence-transformers/paraphrase-multilingual-mpnet-base-v2

```conda install -c conda-forge sentence-transformers```

https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2

In [ ]:
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import torch

start_timer = time.time()

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
mpnet_vectorized_output_array = []

for test_case in edited_output_array:
    # Tokenize
    encoded_input = tokenizer(str(test_case), return_tensors='pt', padding=True, truncation=True, max_length=512)
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, average pooling
    embedding = mean_pooling(model_output, encoded_input['attention_mask'])
    mpnet_vectorized_output_array.append(F.normalize(embedding, p=2, dim=1))

# from sentence_transformers import SentenceTransformer
# 
# start_timer = time.time()
# 
# mpnet_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
# mpnet_vectorized_output_array = []
# 
# for test_case in edited_output_array:
#     mpnet_vectorized_output_array.append(mpnet_model.encode(str(test_case)))

def output_mpnet_to_csv():
    for i in range(len(mpnet_vectorized_output_array)):
        pd.DataFrame(mpnet_vectorized_output_array[i]).to_csv("mpnet_output/mpnet_vectorized_output_" + str(i) + ".csv", index = False)

output_mpnet_to_csv()

end_timer = time.time()
print(end_timer - start_timer)
mpnet_vectorized_output_array

# google-bert/bert-base-multilingual-cased 

https://huggingface.co/google-bert/bert-base-multilingual-cased

In [ ]:
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel
import torch

start_timer = time.time()

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')
bert_vectorized_output_array = []

def bert_vectorize_text(text) -> torch.Tensor:
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)

    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return F.normalize(cls_embedding, p=2, dim=1)

for test_case in edited_output_array:
    bert_vectorized_output_array.append(bert_vectorize_text(str(test_case)))

def output_bert_to_csv():
    for i in range(len(bert_vectorized_output_array)):
        pd.DataFrame(bert_vectorized_output_array[i]).to_csv("bert_output/bert_vectorized_output_" + str(i) + ".csv", index = False)

output_bert_to_csv()

end_timer = time.time()
print(end_timer - start_timer)
bert_vectorized_output_array

# intfloat/multilingual-e5-large-instruct

https://huggingface.co/intfloat/multilingual-e5-large-instruct

In [ ]:
# from sentence_transformers import SentenceTransformer
# 
# start_timer = time.time()
# 
# model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

start_timer = time.time()

def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large-instruct')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large-instruct')
e5_vectorized_output_array = []

for test_case in edited_output_array:
    # Tokenize the input texts
    batch_dict = tokenizer(str(test_case), padding=True, truncation=True, return_tensors='pt', max_length=512)
    with torch.no_grad():
        outputs = model(**batch_dict)

    emb = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    e5_vectorized_output_array.append(F.normalize(emb, p=2, dim=1))


def output_e5_to_csv():
    for i in range(len(e5_vectorized_output_array)):
        pd.DataFrame(e5_vectorized_output_array[i]).to_csv("e5_output/e5_vectorized_output_" + str(i) + ".csv", index = False)

output_e5_to_csv()

end_timer = time.time()
print(end_timer - start_timer)
e5_vectorized_output_array

# ai-forever/sbert_large_nlu_ru

https://huggingface.co/ai-forever/sbert_large_nlu_ru

In [ ]:
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModel
import torch

start_timer = time.time()

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask) -> torch.Tensor:
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")
sbert_vectorized_output_array = []

for test_case in edited_output_array:
    encoded_input = tokenizer(str(test_case), padding=True, truncation=True, return_tensors='pt', max_length=512)
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    emb = mean_pooling(model_output, encoded_input['attention_mask'])
    sbert_vectorized_output_array.append(F.normalize(emb, p=2, dim=1))

def output_sbert_to_csv():
    for i in range(len(sbert_vectorized_output_array)):
        pd.DataFrame(sbert_vectorized_output_array[i]).to_csv("sbert_output/sbert_vectorized_output_" + str(i) + ".csv", index = False)

output_sbert_to_csv()
    
end_timer = time.time()
print(end_timer - start_timer)
sbert_vectorized_output_array

In [ ]:
print((spacy_vectorized_output_array[0][0]).shape)
print((mpnet_vectorized_output_array[0][0]).shape)
print((bert_vectorized_output_array[0][0]).shape)
print((e5_vectorized_output_array[0][0]).shape)
print((sbert_vectorized_output_array[0][0]).shape)

# Оценка качества

Для оценки семантики векторизованных тест-кейсов будет использоваться косинусное сходство.
Как можно обратить внимание ниже, нет никакого смысла считать косинусное сходство одинаковых предложений, обработанных разными эмбединговыми моделями.

In [ ]:
import torch
import torch.nn.functional as F
from typing import List

def count_cosine_similarity(array1: List[torch.Tensor], array2: List[torch.Tensor]):
    result = []
    for i, j in zip(array1, array2):
         result.append(F.cosine_similarity(i, j))

    return result

mpnet_bert = count_cosine_similarity(mpnet_vectorized_output_array, bert_vectorized_output_array)
e5_sbert = count_cosine_similarity(e5_vectorized_output_array, sbert_vectorized_output_array)

print(mpnet_bert)
print(e5_sbert)

Поэтому для начала необходимо посчитать косинусное сходство между разными предложениями внутри одной модели

In [ ]:
def sentence_cosine_similarity(array):
    # first_case = F.cosine_similarity(array[0][0], array[0][1]) #два слова заменены с английского на русский
    # second_case = F.cosine_similarity(array[0][2], array[0][3]) #два одинаковых предложения на русском и английском
    print(array[0][0])

sentence_cosine_similarity(spacy_vectorized_output_array)

# Визуализация результатов

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# 
# #plt_models = ["SpaCy", "paraphrase-multilingual-mpnet-base-v2", "bert-base-multilingual-cased", "sbert-base-multilingual-uncased", "multilingual-e5-large"]
# plt.figure(figsize=(10, 6))
# sns.heatmap(mpnet_bert, annot=True, cmap="viridis", cbar=True)
# plt.title("Cosine Similarity between Queries and Documents")
# plt.xlabel("Documents")
# plt.ylabel("Queries")
# plt.show()

In [ ]:
# sns.heatmap(e5_sbert, annot=True, cmap="viridis", cbar=True)
# plt.title("Cosine Similarity between Queries and Documents")
# plt.xlabel("Documents")
# plt.ylabel("Queries")
# plt.show()